# Emissions to forcing for GHGs + ozone

Keep the AR6 treatment in FaIR v1.6.2+ and remove all previous incarnations.

We'll calculate the GHG concentrations from emissions first, and then use both to calculate ozone forcing along with the temperature feedback. We will then check the calculated forcing against the AR6 time series, which used a very similar method.

Here we are starting to build the final incarnation of the model: we have to handle ERFs from emissions and GHGs.

In [ ]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import warnings

from fair21.forcing.ghg import ghg
from fair21.constants import TIME_AXIS, SPECIES_AXIS
from fair21.constants.gases import MOLWT, burden_per_emission, lifetime
from fair21.defaults import slcf_list, gas_list, montreal_gas_list, minor_gas_list, n_gas_boxes
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon,
    radiative_efficiency
)
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.defaults.species import pre_industrial_emissions

from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha
)
from fair21.gas_cycle.forward import step_concentration

from fair21.constants.gases import BR_ATOMS, CL_ATOMS
from fair21.defaults.ozone import radiative_efficiency as o3_radiative_efficiency, br_cl_ratio, fractional_release
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.defaults.gases import pre_industrial_concentration
from fair21.defaults import gas_list, slcf_list

from fair21.forcing.ozone import thornhill_skeie

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
species_list = gas_list + slcf_list
species_list.append('O3')
species_list

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}
effective_radiative_forcing = {}

n_timesteps = 751
n_scenarios = len(scenarios)

n_configs = 1

# kicker for ozone
n_species = len(species_list)
n_species

In [ ]:
# grab some emissions
species_index_mapping = {}
emissions_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    for ispec, specie in enumerate(species_list):
        species_index_mapping[specie] = ispec
        specie_rcmip_name = specie.replace("-", "")
        try:
            emissions_array[:, iscen, 0, ispec, 0] = df.loc[
                (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+specie_rcmip_name)) & (df['Region']=='World'), '1750':
            ].interpolate(axis=1).values.squeeze()
        except ValueError:
            warnings.warn("{} was not found in emissions file".format(specie))
    
        # CO2 and N2O units need to behave
        if specie in ('CO2', 'N2O'):
            emissions_array[:, iscen, :, ispec, :] = emissions_array[:, iscen, :, ispec, :] / 1000

In [ ]:
species_index_mapping
emissions_array[265, 7, 0, :, 0]

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp585-mean-temperature.csv')
ssp585_temperature_rfmip = df['temperature'].values

In [ ]:
radiative_efficiency

In [ ]:
# initialise arrays. Using arrays makes things run quicker

# old: scen, species, time, box
# new: time, scen, config, species, box

# where we don't care about keeping outputs we use a singleton dimension to maintain bookkeeping
# possible memory saver: differentiate between ensembles where we vary emissions and those where we vary climate parameters

n_configs = 1

# this scenario definition needs to be part of the initialisation, either from pyam, scmdata or fair emissions files
concentration_array = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan
g0 = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
g1 = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
alpha_lifetime = np.ones((1, n_scenarios, n_configs, n_species, 1))
airborne_emissions = np.zeros((1, n_scenarios, n_configs, n_species, 1))
gas_boxes = np.zeros((1, n_scenarios, n_configs, n_species, n_gas_boxes))
iirf_0_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
iirf_cumulative_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
iirf_temperature_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
iirf_airborne_array = np.ones((1, n_scenarios, n_configs, n_species, 1)) * np.nan
burden_per_emission_array = np.ones((1, 1, 1, n_species, 1)) * np.nan
lifetime_array = np.ones((1, 1, 1, n_species, n_gas_boxes)) * np.nan
pre_industrial_emissions_array = np.ones((1, n_scenarios, 1, n_species, 1)) * np.nan     # scenario axis, config or both?
pre_industrial_concentration_array = np.ones((1, n_scenarios, 1, n_species, 1)) * np.nan
partition_fraction_array = np.zeros((1, n_scenarios, 1, n_species, n_gas_boxes))
natural_emissions_adjustment_array = np.ones((1, n_scenarios, 1, n_species, 1)) * np.nan

cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)

# this one for the general forcing
tropospheric_adjustment_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
radiative_efficiency_array = np.ones((1, 1, n_configs, n_species, 1)) * np.nan
erf = np.ones((n_timesteps, n_scenarios, n_configs, n_species, 1)) * np.nan

# ozone
fractional_release_array = np.zeros((1, 1, 1, n_species, 1))
cl_atoms_array = np.zeros((1, 1, 1, n_species, 1))
br_atoms_array = np.zeros((1, 1, 1, n_species, 1))
o3_radiative_efficiency_array = np.zeros((1, 1, n_configs, n_species, 1))

for ispec, specie in enumerate(species_list):
    # options pertaining to SLCFs
    if specie in slcf_list:
        pre_industrial_emissions_array[..., ispec, :] = pre_industrial_emissions[specie]
    
    # options pertaining to GHGs
    if specie in gas_list:
        lifetime_array[..., ispec, :] = lifetime[specie]
        partition_fraction_array[..., ispec, :] = partition_fraction[specie]
        iirf_0_array[..., ispec, :] = iirf_0[specie]
        iirf_cumulative_array[..., ispec, :] = iirf_cumulative[specie]
        iirf_temperature_array[..., ispec, :] = iirf_temperature[specie]
        iirf_airborne_array[..., ispec, :] = iirf_airborne[specie]
        burden_per_emission_array[..., ispec, :] = burden_per_emission[specie]
        pre_industrial_concentration_array[..., ispec, :] = pre_industrial_concentration[specie]
        partition_fraction_array[..., ispec, :] = partition_fraction[specie]
        natural_emissions_adjustment_array[..., ispec, :] = natural_emissions_adjustment[specie]
        fractional_release_array[..., ispec, :] = fractional_release[specie]
        br_atoms_array[..., ispec, :] = BR_ATOMS[specie]
        cl_atoms_array[..., ispec, :] = CL_ATOMS[specie]
        
    # options pertaining only to minor GHGs
    if specie in minor_gas_list:
        radiative_efficiency_array[..., ispec, :] = radiative_efficiency[specie]
        

    # options pertaining to all species
    tropospheric_adjustment_array[..., ispec, :] = tropospheric_adjustment[specie]
    o3_radiative_efficiency_array[..., ispec, :] = o3_radiative_efficiency[specie]

g0, g1 = calculate_g(lifetime_array, partition_fraction_array)

In [ ]:
o3_radiative_efficiency

In [ ]:
%%time

for i_timestep in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_array[[i_timestep], ...],
        airborne_emissions[[0], ...],
        ssp585_temperature_rfmip[i_timestep],
        iirf_0_array,
        iirf_cumulative_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    concentration_array[[i_timestep], ...], gas_boxes, airborne_emissions = step_concentration(
        emissions_array[[i_timestep], ...], 
        gas_boxes,
        airborne_emissions, 
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array,
    )

In [ ]:
concentration_array[265,6,0,:,0]

In [ ]:
%%time
# Greenhouse gases
erf[..., list(species_index_mapping.values()), :] = ghg(
    concentration_array[..., list(species_index_mapping.values()), :],
    pre_industrial_concentration_array[..., list(species_index_mapping.values()), :],
    tropospheric_adjustment_array[..., list(species_index_mapping.values()), :], 
    radiative_efficiency_array[..., list(species_index_mapping.values()), :],
    species_index_mapping
)

In [ ]:
%%time
erf[..., [species_index_mapping["O3"]], :] = thornhill_skeie(
    emissions_array,
    concentration_array,
    pre_industrial_emissions_array,
    pre_industrial_concentration_array,
    fractional_release_array,
    cl_atoms_array,
    br_atoms_array,
    tropospheric_adjustment_array,
    o3_radiative_efficiency_array,
    species_index_mapping,
    temperature=ssp585_temperature_rfmip[None, :, None, None, None]
)

In [ ]:
ar6_df = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

In [ ]:
# to double check, ensure we are picking up everything we want from AR6
pl.plot(erf[:, :, 0, 50, 0])
pl.plot(np.arange(270), ar6_df['o3'], color='k', label='AR6')
pl.legend()